In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import os
import random
import numpy as np
import sys
from multiprocessing import Pool
import gc
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import random
import gzip
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import lit, col, when
from pyspark.sql.types import StringType
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
spark = SparkSession.builder.master('local[*]').config("spark.driver.memory", "100g").appName('Dataframe').getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")
spark.conf.set('spark.sql.debug.maxToStringFields',100000)

In [ ]:
files= os.listdir('/s3data')
lst = ['/s3data/'+i for i in files]
del files
gc.collect()
samples = random.sample(lst,35700)
del lst
gc.collect()

In [4]:
%%time
df = spark.read.json(samples)
del samples
gc.collect()

In [ ]:
df = df.dropDuplicates()

In [ ]:
lst = [i.replace('.','_') for i in df.columns]

In [ ]:
df = df.toDF(*lst)
del lst
gc.collect()

In [ ]:
feat = pd.read_excel('/home/rajatsharma/DSA/lcols.xlsx')
lst = [i.replace('.','_') for i in feat['Cols']]

In [ ]:
for i in lst:
    if i not in df.columns:
        df = df.withColumn(i, lit(None).cast(StringType()))

In [ ]:
data = df.select(*lst)
del lst,df
gc.collect()

In [ ]:
# data.filter("event_type is null").count()
data = data.where(col('event_type').isNotNull())

In [ ]:
data.groupBy('alert_category').count().show()

In [ ]:
data.fillna('not_alert',subset=['alert_category','alert_signature']).count()

In [ ]:
to_replace = ['layer_tcp_tcp_flags_ack','layer_tcp_tcp_flags_cwr', 'layer_tcp_tcp_flags_ecn',
              'layer_tcp_tcp_flags_fin','layer_tcp_tcp_flags_push','layer_tcp_tcp_flags_reset',
              'layer_tcp_tcp_flags_syn','layer_tcp_tcp_flags_urg','layer_ip_ip_flags_df',
               'layer_ip_ip_flags_mf']

In [ ]:
for i in to_replace:
    data = data.withColumn(i,when((col(i)==1.0),True))
    data = data.withColumn(i,when((col(i)==0.0),False))


In [ ]:
data.replace({'None': None})

In [ ]:
data.fillna(False,subset=['layer_tcp_tcp_options_sack_perm'])
data = data.withColumn('layer_tcp_tcp_options_sack_perm',when((col('layer_tcp_tcp_options_sack_perm')=='04:02'),True))

In [ ]:
data.groupBy('event_type').count().show()

In [ ]:
data.to_csv('home/rajatsharma/35k.csv',index = False)